In [1]:
from transformers import GPT2LMHeadModel

/Users/reza/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_hf = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
list(model_hf.modules())[0].transformer.h[0]

In [10]:
model_hf

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
for k,v in model_hf.state_dict().items():
    print(k, v.shape)

In [ ]:
sd_hf = model_hf.state_dict()
sd_keys_hf = sd_hf.keys()
# for k in sd_keys_hf:
#     print(k.endswith('.attn.bias'))
sd_keys_hf

In [5]:
import torch
from torch.nn import functional as F

In [14]:
x = torch.randint(0, 50000, (4, 8))
x
logits = model_hf(x)

In [15]:
logits.logits.shape

torch.Size([4, 8, 50257])

In [21]:
logits.logits.view(-1, 50257).shape

torch.Size([32, 50257])

In [24]:
x.view(-1).shape

torch.Size([32])

In [ ]:
list(model_hf(x).items())[0][1].shape

In [ ]:
max_length = 50
num_return_sequences = 1

import tiktoken
enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode("Zebras are classified in the genus Equus (known as equines) along with horses")
tokens = torch.tensor(tokens, dtype=torch.long)
tokens = tokens.unsqueeze(0)
tokens = tokens.repeat(num_return_sequences,1)
x = tokens
x

while x.size(1) < max_length:
    with torch.no_grad():
        logits = model_hf(x).logits
        logits = logits[:, -1, :]
        probs = F.softmax(logits, dim=-1)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
        ix = torch.multinomial(topk_probs, 1)
        xcol = torch.gather(topk_indices, -1, ix)
        x = torch.cat((x, xcol), dim=1)

        for i in range(num_return_sequences):
            tokens = x[i, :].tolist()
            decoded = enc.decode(tokens)
            print(decoded)


In [ ]:
logits = model_hf(tokens)
logits = logits.logits

In [ ]:
logits = logits[:, -1, :] # (B, C)

In [ ]:
logits.shape

In [ ]:
probs = F.softmax(logits, dim=-1)
probs

In [ ]:
# x = x.tolist()
enc.decode(x[0])

In [9]:
model_hf.generate(x)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[36439, 34975, 18764, 11703, 30391, 21332, 28976, 13361,    13,  8457,
           198,   198,   464,  1708,   318,   257,  1351,   286,   477,   262,
         11933, 12782,   326,   389,  1695,   329,   779,   351],
        [11065,  4212, 49673,  5496, 22674, 20194, 44449,  2761,    11,   290,
           262,  1109,   326,   262,  5496,   318,   407,   257,  5496,    13,
           198,   198,   464,  5496,   318,   407,   257,  5496],
        [ 5807, 40841, 46574, 26804, 30433,  7710,   305, 19938,   284,   262,
         40841,  5402,    13,   198,   198,   464,  5402,   373, 20325,   416,
           262,  2679,    12,  3106,  1448,    11,   262,  2679],
        [38114, 40623, 31736, 15685, 28000,  1153, 31412, 33938,    11,   314,
          1101,   407,  1654,   611,   314,  1101,  1016,   284,   307,  1498,
           284,   466,   428,    13,   314,  1101,   407,  1654]])